In [1]:
import pandas as pd
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:

df = pd.read_excel('NFLX.xlsx')

In [4]:
df.head

<bound method NDFrame.head of           Date   Open     High    Low    Close  Volume
0   2016-01-04  111.55  111.75  105.21  110.15  188288
1   2016-01-05  109.01  111.20  105.85  107.67  160849
2   2016-01-06  105.45  118.45  104.50  118.25  307273
3   2016-01-07  118.30  122.18  112.29  114.45  311756
4   2016-01-08  116.00  117.72  111.00  111.01  164152
5   2016-01-11  111.82  116.79  111.20  115.08  203389
6   2016-01-12  115.28  117.78  115.08  116.85  139374
7   2016-01-13  116.62  117.25  104.53  106.25  226858
8   2016-01-14  105.28  109.25  101.21  107.00  217312
9   2016-01-15  106.66  106.66  101.64  104.07  177376
10  2016-01-19  106.00  123.04  103.00  115.30  294243
11  2016-01-20  114.23  115.00   97.05  108.11  489010
12  2016-01-21  107.10  109.60  101.31  101.41  288271
13  2016-01-22  104.35  105.10   99.22  100.75  247316
14  2016-01-25  100.02  102.68   98.50   98.60  185782
15  2016-01-26   97.69  100.55   94.85   96.91  203087
16  2016-01-27   96.99   97.85   90

In [6]:



# Modelling CAPM and calculating alpha and beta from historical values.
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def capm(start_date, end_date, ticker1, ticker2):
    df = pdr.get_data_yahoo(ticker1, start_date, end_date)
    dfb = pdr.get_data_yahoo(ticker2, start_date, end_date)

# create a time-series
    rts = df.resample('M').last()
    rbts = dfb.resample('M').last()
    dfsm = pd.DataFrame({'s_open' : rts['Open'], 'b_open' : rbts['Open']}, index=rts.index)

# compute returns
    dfsm[['s_returns', 'b_returns']] = dfsm[['s_open','b_open']]/dfsm[['s_open','b_open']].shift(1) -1
    dfsm = dfsm.dropna()

    covmat = np.cov(dfsm["s_returns"], dfsm["b_returns"])

# calculate measures now
    beta = covmat[0,1]/covmat[1,1]
    alpha = np.mean(dfsm["s_returns"])-beta*np.mean(dfsm["b_returns"])

# r_squared     = 1.0 - SS_res/SS_tot
    y = beta * dfsm["b_returns"] + alpha
    SS_res = np.sum(np.power(y - dfsm["s_returns"],2))
    SS_tot = covmat[0,0]*(len(dfsm) - 1) # SS_tot is sample_variance*(n-1)

    r_squared = 1.0 - SS_res/SS_tot
# Volatility for the full time and 1-year momentum
    volatility = np.sqrt(covmat[0,0])
    momentum = np.prod(1+dfsm["s_returns"].tail(12).values) - 1.0


    prd = 12.0 # used monthly returns; 12 periods to annualize
    alpha = alpha*prd
    volatility = volatility*np.sqrt(prd)

    print ("Beta, alpha, r_squared, volatility, momentum:")
    print (beta, alpha, r_squared, volatility, momentum)

    %pylab
    fig,ax = plt.subplots(1,figsize=(20,10))
    ax.scatter(dfsm["b_returns"], dfsm['s_returns'], label="Data points")
    beta,alpha = np.polyfit(dfsm["b_returns"], dfsm['s_returns'], deg=1)
    ax.plot(dfsm["b_returns"], beta*dfsm["b_returns"] + alpha, color='red', label="CAPM line")


    plt.title('Capital Asset Pricing Model, finding alphas and betas')
    plt.xlabel('Market return $R_m$', fontsize=14)
    plt.ylabel('Stock return $R_i$')
#     plt.text(0.05, 0.05, r'$R_i = \beta * R_m + \alpha$', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()
    return beta, y


beta, y = capm('2016-01-04', '2018-08-30','NFLX', '^GSPC')




Beta, alpha, r_squared, volatility, momentum:
1.15745357846 0.437762372699 0.0453957660426 0.434911477145 1.11262473702
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\incha\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
